<a href="https://colab.research.google.com/github/ravishanilneth/Early-Detection-of-Alzheimer-s-Disease-Using-Explainable-AI-Powered-Deep-Learning/blob/main/MRI_Preprocessing_in_nifti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nibabel

  Using cached nibabel-5.2.1-py3-none-any.whl (3.3 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from skimage import exposure
from skimage.filters import threshold_otsu
from skimage.morphology import binary_erosion, binary_dilation
from skimage.measure import label, regionprops
from skimage.transform import resize
from scipy.ndimage import zoom

# Function to preprocess MRI images for Alzheimer's disease detection
def preprocess_mri(nii_file_path, output_file_path):
    # Load the .nii file
    img = nib.load(nii_file_path)
    img_data = img.get_fdata()

    # Intensity normalization
    img_data = exposure.rescale_intensity(img_data)

    # Skull stripping
    threshold = threshold_otsu(img_data)
    binary_mask = img_data > threshold
    labeled_mask = label(binary_mask)
    largest_region = max(regionprops(labeled_mask), key=lambda r: r.area)
    binary_mask = labeled_mask == largest_region.label
    img_data[~binary_mask] = 0

    # Resize the image to a common resolution
    target_shape = (128, 128, 128)
    img_data = zoom(img_data,
                    (target_shape[0] / img_data.shape[0],
                     target_shape[1] / img_data.shape[1],
                     target_shape[2] / img_data.shape[2]))

    # Save the preprocessed image
    nib.save(nib.Nifti1Image(img_data, img.affine), output_file_path)

# Path to the main folder containing subfolders for each patient
main_folder = "D:/KDU/final year research/Alzhimer's disease early detection/SUPIRI DATASETS/OASIS1/OASIS_MNI_Registered-main"

# Output folder to save preprocessed images
output_folder = "C:/Users/RAVIYA/Desktop/preprocessed output"

# Iterate through subfolders for each patient
for root, dirs, files in os.walk(main_folder):
    for file in files:
        if file.endswith('.nii'):
            nii_file_path = os.path.join(root, file)
            # Create the corresponding output path
            relative_path = os.path.relpath(root, main_folder)
            output_subfolder = os.path.join(output_folder, relative_path)
            os.makedirs(output_subfolder, exist_ok=True)
            output_file_path = os.path.join(output_subfolder, file)
            preprocess_mri(nii_file_path, output_file_path)

print("Preprocessing completed successfully.")


Preprocessing completed successfully.
